In [1]:
# Installing required libraries for the dtmf encoding code
!pip install keyboard
!pip install sounddevice
!pip install pygame

In [2]:
#Encoding 
import numpy as np
import sounddevice as sd
import pygame
import threading
import keyboard
import time

# -----------------------------
# Initialize Pygame for Joystick Input
# -----------------------------
pygame.init()
if pygame.joystick.get_count() == 0:
    print("No joystick detected. Exiting.")
    pygame.quit()
    exit()
joystick = pygame.joystick.Joystick(0)
joystick.init()
print("Joystick initialized:", joystick.get_name())

# -----------------------------
# DTMF Frequency Tables
# -----------------------------
low_freqs = [697, 697, 697, 697, 770, 770, 770, 770,
             852, 852, 852, 852, 941, 941, 941, 941]
high_freqs = [1209, 1336, 1477, 1633, 1209, 1336, 1477, 1633,
              1209, 1336, 1477, 1633, 1209, 1336, 1477, 1633]
hex_keys = list("0123456789ABCDEFHT")
HEAD = "H"
TAIL = "T"
SPECIAL_H_LOW, SPECIAL_H_HIGH = 500, 2000
SPECIAL_T_LOW, SPECIAL_T_HIGH = 300, 2500
Fs = 8000  # Sampling frequency

message_duration = 0.4  # 400 ms per message
num_symbols = 5  # H, M1, M2, M3, T
symbol_duration = message_duration / num_symbols  # 80 ms per symbol
t_tone = np.arange(0, symbol_duration, 1 / Fs)

# Joystick control variables
M1, M2, M3 = 128, 128, 128
M1_speed, M2_speed, M3_speed = 0, 0, 0
decay_factor = 0.98
max_speed = 5

def adjust_axis_value(raw_value, deadzone=0.1, exponent=3):
    """Apply deadzone and adjust sensitivity of joystick input."""
    if abs(raw_value) < deadzone:
        return 0.0
    return np.sign(raw_value) * (abs(raw_value) ** exponent)

def axis_to_speed(axis_value):
    """Convert joystick axis value to speed."""
    return int(axis_value * max_speed)

def generate_tone(symbol):
    """Generate the DTMF tone for a given symbol."""
    if symbol == HEAD:
        f1, f2 = SPECIAL_H_LOW, SPECIAL_H_HIGH
    elif symbol == TAIL:
        f1, f2 = SPECIAL_T_LOW, SPECIAL_T_HIGH
    else:
        index = hex_keys.index(symbol)
        f1, f2 = low_freqs[index], high_freqs[index]

    return np.sin(2 * np.pi * f1 * t_tone) + np.sin(2 * np.pi * f2 * t_tone)

def send_dtmf_message():
    """Generates and plays continuous DTMF messages without any gaps."""
    global M1, M2, M3, M1_speed, M2_speed, M3_speed

    while not keyboard.is_pressed("q"):
        pygame.event.pump()

        # Read joystick values and update speed
        raw_m1 = joystick.get_axis(1)
        raw_m2 = joystick.get_axis(0)
        raw_m3 = joystick.get_axis(3)
        M1_speed = axis_to_speed(adjust_axis_value(raw_m1))
        M2_speed = axis_to_speed(adjust_axis_value(raw_m2))
        M3_speed = axis_to_speed(adjust_axis_value(raw_m3))

        # Update M1, M2, M3 values based on speed
        M1 = max(0, min(255, M1 + M1_speed))
        M2 = max(0, min(255, M2 + M2_speed))
        M3 = max(0, min(255, M3 + M3_speed))

        # Apply gradual slowdown when joystick is released
        M1_speed *= decay_factor
        M2_speed *= decay_factor
        M3_speed *= decay_factor

        # Convert to HEX for transmission
        m1_hex = format(M1, '02X')
        m2_hex = format(M2, '02X')
        m3_hex = format(M3, '02X')
        full_sequence = HEAD + m1_hex + m2_hex + m3_hex + TAIL
        print(f"Sending Message: {full_sequence}")

        # Generate the waveform for the full message (concatenating all tones)
        message_waveform = np.concatenate([generate_tone(symbol) for symbol in full_sequence])

        # Play the message immediately (ensuring no gaps)
        t0 = time.time()
        sd.play(message_waveform, Fs, loop=False)
        print(time.time() - t0)
        # Wait for the message to finish before generating the next one
          
        sd.wait()
# Start continuous transmission thread
transmission_thread = threading.Thread(target=send_dtmf_message, daemon=True)
transmission_thread.start()

print("Press Q to quit.")
while not keyboard.is_pressed("q"):
    pass  # Keep program running

pygame.quit()
print("Program terminated.")

pygame 2.6.1 (SDL 2.28.4, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
Joystick initialized: PS4 Controller
Press Q to quit.
Sending Message: H808080T
0.11617469787597656
Sending Message: H808080T
0.12674736976623535
Sending Message: H7B8080T
0.2215261459350586
Sending Message: H7B8480T
0.22649097442626953
Sending Message: H768480T
0.21122384071350098
Sending Message: H768480T
0.23944854736328125
Sending Message: H768480T
0.23445844650268555
Program terminated.


Exception in thread Thread-5 (send_dtmf_message):
Traceback (most recent call last):
  File "C:\Users\96653\anaconda31\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\96653\anaconda31\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\96653\AppData\Local\Temp\ipykernel_22728\2123798481.py", line 73, in send_dtmf_message
pygame.error: video system not initialized
